# **Students' Dropout Prediction*



Introduction


This project aimed to contribute to the reduction of dropouts in higher education by using machine learning techniques. It essentially identifies students at risk of dropping out early in their academic path, allowing support strategies to be implemented. 
The dataset used for this project is from a higher education institution in Portugal that contains information of students at the time of enrolment like demographics, socio-economic factors along with their academic performance during the first year(2 semesters). There is also the status of the student(Droput/Enrolled/Graduate) for each instance.For this project, we customized the target variable as binary classification model (Drop-out or Not Drop-out).

After necessary data cleaning and preprocessing, Classification models will be trained and evaluated using the relevant machine learning algorithms. 


Our prediction task is to determine whether the student will drop-out or not based on various attributes like 
Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nationality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate, Inflation rate and GDP).


## Step 1: Install and/or import necessary packages

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np
# Plot Handling/Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Preprocessing data
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,RandomizedSearchCV
# MODELS
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Performance measures
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,fbeta_score,make_scorer,confusion_matrix, classification_report,plot_roc_curve, roc_auc_score, roc_curve, auc, RocCurveDisplay, PrecisionRecallDisplay, precision_recall_curve


## Step 2: Preliminary Business Problem Scoping

We are developing a binary classifier to identify if a student is Dropout or not based on the academic performance in the first year and the available demographic information. Below are the various scenarios that are 
anticipated during the model training:

True Positives(TP)- When model predicts the student as "Drop-out" and the actual is also "Drop-out".
True Negatives(TN)- When model predicts the student as "Not Drop-out" and the actual is "Not Drop-out" as well.
False Positives(FP)-When model predicts the student as "Drop-out" when in reality the student is "Not Drop-out".
False Negatives(FN)-When model predicts the student as "Not Drop-out" when in reality the student is "Drop-out".


The model is essentially being developed to solve the drop-out problem in the university wherein the University can intervene with programs like career-couselling,mentorship,alternate class scheduling, online classes etc., to encourage the student complete the course. 
Although, there is a cost for the university (in terms of the additional resources it will have to spend) if there are False Postives,it is relatively less in comparision with False Negatives. The revenue loss due to "drop-outs" far outweighs the cost of additional resources for the interventions by the university.

So, our approach will be to reduce the FNs as much as possible(ideally make them zero). In other words, we aim to maximize the "Recall Score" in the project. However, if there is data imbalance, we will focus on maximizing the "F2 score" which considers both precision and recall with a higher weightage to 'Recall'. The model with high Recall/F2 score is the better performing model.

We will carry out the analysis in two cases. In Case-1, we are intereseted to check whether we can predict the drop-outs only from the admission information(i.e.,attributes collected at the time of enrolment of the students). If we are successful(i.e, if we get good performance scores), the university can engage with the students right from the beginning of the semester. In Case-2, we will carry out the analysis based on the entire data excluding the irrelevant attributes. 


## Step 3: Load, clean and prepare data for analysis

In [ ]:
df = pd.read_csv(r'C:\Users\shilp\Downloads\Student_data_Mac_2 (1).csv')

### Explore the data

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

**From the above, we can see that there are no null values in the dataset**

In [ ]:
df.isna().sum()

## Step 4: Clean and transform data

**Clean the column names**

In [ ]:
df.columns = [s.strip().replace(' ', '_') for s in df.columns] 

In [ ]:
df = df.rename(columns={'Nacionality': 'Nationality'})


In [ ]:
df['Target'].unique()

**Transform multi-class to binary class**

In [ ]:
status = []

for i in df['Target']:
    if i == 'Dropout':
        status.append('Drop-out')
    else:
        status.append('Not Drop-out')
        

**Create a new column with binary class values and drop the multi-class column from the dataframe**

In [ ]:
df['status'] = status

In [ ]:
df.status.unique()

In [ ]:
df = df.drop(columns = ['Target'])

In [ ]:
category_cols = ['Marital_status','Application_mode','Application_order','Course','Daytime/evening_attendance'
                 ,'Previous_qualification','Nationality',"Mother's_qualification","Father's_qualification"
                 ,"Mother's_occupation","Father's_occupation",'Displaced','Educational_special_needs'
                 ,'Debtor','Tuition_fees_up_to_date','Gender','Scholarship_holder'
                 ,'International']

**Encode the Target variable**

In [ ]:
df.status = df.status.replace('Not Drop-out', 0, regex=True)
df.status = df.status.replace('Drop-out', 1, regex=True)  

**Transform the datatypes to categorical**

In [ ]:
for col in category_cols:
    df[col] = df[col].astype('category')

In [ ]:
df.status = df.status.astype('category')

In [ ]:
df.head()

Note:

There is an ambiguity on the year considered(whether it is during enrolment time/at the time of drop-out)
for the macro-economic factors in the dataset ("Employment rate", "GDP" and "Inflation). 
Hence, the same are dropped for the analysis.


# Case-1: Prediction with only the student data available at the time of enrolment 

(i.e, removing the attributes-student academic performance at the university and macro-economic factors from the analysis)


**Create a dataframe(df_1) with the student attributes available at the time of enrolment**

In [ ]:

df_1 = df[['Marital_status', 'Application_mode', 'Application_order', 'Course',
       'Daytime/evening_attendance', 'Previous_qualification',
       'Previous_qualification_(grade)', 'Nationality',
       "Mother's_qualification", "Father's_qualification",
       "Mother's_occupation", "Father's_occupation", 'Admission_grade',
       'Displaced', 'Educational_special_needs', 'Debtor',
       'Gender', 'Scholarship_holder',
       'Age_at_enrollment', 'International','status'
        ]]


**Encoding dummies for the categorical variables**

In [ ]:
cat_cols = [col for col in df_1.columns if df_1[col].dtype in ['category']]

cat_cols.remove('status')


In [ ]:

df_1 = pd.get_dummies(df_1, columns = cat_cols)


df_1.head()


**Visualize the class counts to check for imbalance**

In [ ]:
sns.countplot(df_1.status)

**Partition the data into Training and Test sets for model evaluation**

In [ ]:

train_df, test_df = train_test_split(df_1, train_size=0.7, random_state=99)
#We've decided to utilize a training/test split of the data at 70% training and 30% testing. This percentage split ratio is inline with common practice for small to medium sized datasets, which this data represents. Moreover, we have decided not to do a three way data split, as we are only testing two models and we wish to allocate as much data as possible to training.



In [ ]:

## Step-6: Address Data Imbalances with Oversampling technique

count_0 = (train_df.status==0).sum()
count_1 = (train_df.status==1).sum()

train_df_0 = train_df.loc[train_df.status==0]
train_df_1 = train_df.loc[train_df.status==1]

train_df_1_oversampled = train_df_1.sample((count_0-count_1),replace=True)

train_df = pd.concat([train_df,train_df_1_oversampled],ignore_index = True)


In [ ]:

sns.countplot(train_df['status'])



In [ ]:
X_train = train_df.drop(columns=['status'])
y_train = train_df['status']
X_test = test_df.drop(columns=['status'])
y_test = test_df['status']

### Training the data on Decision Tree model to ascertain the performance

In [ ]:
dtree=DecisionTreeClassifier(random_state=1)
dtree.fit(X_train, y_train)

In [ ]:
y_pred_DT = dtree.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred_DT)

In [ ]:
DT_scores = []
DT_scores.append({ 'Model' : 'Decision Tree',
                'Accuracy': accuracy_score(y_test, y_pred_DT),
                'Precision': precision_score(y_test, y_pred_DT),
                'Recall':recall_score(y_test, y_pred_DT),
                'F1': f1_score(y_test,y_pred_DT),
                'F2': fbeta_score(y_test,y_pred_DT, beta = 2, average = 'binary')
})

In [ ]:
DT_scores

### Training the data on Random Forest model to ascertain the performance

In [ ]:
rforest = RandomForestClassifier(random_state=99)
_ = rforest.fit(X_train, y_train)

In [ ]:
y_pred_RF = rforest.predict(X_test)
confusion_matrix(y_test,y_pred_RF)

In [ ]:
RF_scores = []
RF_scores.append({ 'Model' : 'Random Forest',
                'Accuracy': accuracy_score(y_test, y_pred_RF),
                'Precision': precision_score(y_test, y_pred_RF),
                'Recall':recall_score(y_test, y_pred_RF),
                'F1': f1_score(y_test,y_pred_RF),
                'F2': fbeta_score(y_test,y_pred_RF, beta = 2, average = 'binary')
})


In [ ]:
RF_scores

### Conclusion for Case-1

Clearly from the above decision tree and Random forest model performance metrics,we can infer that the students' attributes at the time of enrolment has less predictive ability on the drop-out status.

# Case-2 : Model Prediction excluding the macro-economic data attributes like GDP, Unemployment rate and Inflation


Note:
    
There is ambiguity on the year considered (whether it is at the time of enrolment /at the time of student drop-out)
for the macro-economic factors in the dataset("Employment rate", "GDP" and "Inflation).
Hence, the same are being dropped for the analysis.

In [ ]:
df = df.drop(df.columns[[33,34,35]], axis=1) 

**Encoding dummies for the categorical variables**

In [ ]:
df = pd.get_dummies(df, columns = category_cols)
df.head()

**Visualize the class counts to check for imbalance**

In [ ]:
sns.countplot(df['status'])

## Step-5: Partition the data into Training and Test sets for model evaluation

We've decided to utilize a training/test split of the data at 70% training and 30% testing. This percentage split ratio is inline with common practice for small to medium sized datasets, which this data represents. Moreover, we have decided not to do a three way data split as we wish to allocate as much data as possible to training.

In [ ]:
train_df, test_df = train_test_split(df, train_size=0.7, random_state=99)

## Step-6: Address Data Imbalances of training set with Oversampling technique

In [ ]:
count_0 = (train_df.status==0).sum()
count_1 = (train_df.status==1).sum()

In [ ]:
train_df_0 = train_df.loc[train_df.status==0]
train_df_1 = train_df.loc[train_df.status==1]

In [ ]:
train_df_1_oversampled = train_df_1.sample((count_0-count_1),replace=True)

In [ ]:
train_df = pd.concat([train_df,train_df_1_oversampled],ignore_index = True)

In [ ]:
sns.countplot(train_df['status'])

In [ ]:
X_train = train_df.drop(columns=['status'])
y_train = train_df['status']
X_test = test_df.drop(columns=['status'])
y_test = test_df['status']

# Step-7: Train the models

### 7(a) Decision Tree with Hyper-Parametric Tuning

In [ ]:
# Criterion used to guide data splits
criterion = ['gini', 'entropy','log_loss']

max_depth = [int(x) for x in np.linspace(900, 1200, 50)]
max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(0, 100, 20)]
min_samples_leaf = [int(x) for x in np.linspace(1, 100, 30)] 
max_leaf_nodes = [int(x) for x in np.linspace(2, len(y_test), 50)]
max_leaf_nodes.append(None)
min_impurity_decrease = [x for x in np.arange(0.0, 0.01, 0.0001).round(5)]


In [ ]:
# Create the random grid
param_grid_random = { 'criterion': criterion,
                      'max_depth': max_depth,
                      'min_samples_split': min_samples_split,
                      'min_samples_leaf' : min_samples_leaf,
                      'max_leaf_nodes' : max_leaf_nodes,
                      'min_impurity_decrease' : min_impurity_decrease,
                     }


In [ ]:
best_random_search_model = RandomizedSearchCV(
        estimator=DecisionTreeClassifier(random_state=99), 
        scoring='recall', 
        param_distributions=param_grid_random, 
        n_iter = 1500, 
        cv=10, 
        verbose=0, 
        n_jobs = -1
    )
_ = best_random_search_model.fit(X_train, y_train)

In [ ]:
random_search_best_params = best_random_search_model.best_params_
print('Best parameters found:', random_search_best_params)

In [ ]:
param_grid = {
    'max_depth': [920,925,930], 
    'min_samples_split': [24,26,28], 
    'min_samples_leaf' : [1,2,3],
    'min_impurity_decrease': [0.0004, 0.0005,0.0006], 
    'criterion': ['gini']
}

In [ ]:
best_grid_search_model = GridSearchCV(estimator=DecisionTreeClassifier(random_state=99), 
                                    scoring='recall', param_grid=param_grid, cv=10, verbose=0,  n_jobs = -1)
_ = best_grid_search_model.fit(X_train, y_train)

In [ ]:
y_pred_DT = best_grid_search_model.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred_DT)

In [ ]:
DT_scores = []
DT_scores.append({ 'Model' : 'Decision Tree',
                'Accuracy': accuracy_score(y_test, y_pred_DT),
                'Precision': precision_score(y_test, y_pred_DT),
                'Recall':recall_score(y_test, y_pred_DT),
                'F1': f1_score(y_test,y_pred_DT),
                'F2': fbeta_score(y_test,y_pred_DT, beta = 2, average = 'binary')
})

In [ ]:
Decision_Tree_scores= pd.DataFrame(DT_scores)
Decision_Tree_scores

### 7(b) Random Forest- Ensemble Technique(Bagging)

In [ ]:
rforest = RandomForestClassifier(random_state=99)
_ = rforest.fit(X_train, y_train)

In [ ]:
y_pred_RF = rforest.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred_RF)

In [ ]:
RF_scores = []
RF_scores.append({ 'Model' : 'Random Forest',
                'Accuracy': accuracy_score(y_test, y_pred_RF),
                'Precision': precision_score(y_test, y_pred_RF),
                'Recall':recall_score(y_test, y_pred_RF),
                'F1': f1_score(y_test,y_pred_RF),
                'F2': fbeta_score(y_test,y_pred_RF, beta = 2, average = 'binary')
})

In [ ]:
Random_Forest_scores= pd.DataFrame(RF_scores)
Random_Forest_scores

### 7(c) AdaBoost

In [ ]:
aboost = AdaBoostClassifier(random_state=99)
_ = aboost.fit(X_train, y_train)

In [ ]:
y_pred_ADB = aboost.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred_ADB)

In [ ]:
ADB = []
ADB.append({ 
                'Model' : 'AdaBoost',
                'Accuracy': accuracy_score(y_test, y_pred_ADB),
                'Precision': precision_score(y_test, y_pred_ADB),
                'Recall':recall_score(y_test, y_pred_ADB),
                'F1': f1_score(y_test,y_pred_ADB),
                'F2': fbeta_score(y_test,y_pred_ADB, beta = 2, average = 'binary')
})

In [ ]:
AdaBoost_scores= pd.DataFrame(ADB)
AdaBoost_scores

### 7(d) Gradient Boost

In [ ]:
gboost = GradientBoostingClassifier(random_state=99)
_ = gboost.fit(X_train, y_train)


In [ ]:
y_pred_GB = gboost.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred_GB)

In [ ]:
GB = []
GB.append({     'Model' : 'Gradient Boost',
                'Accuracy': accuracy_score(y_test, y_pred_GB),
                'Precision': precision_score(y_test, y_pred_GB),
                'Recall':recall_score(y_test, y_pred_GB),
                'F1': f1_score(y_test,y_pred_GB),
                'F2': fbeta_score(y_test,y_pred_GB, beta = 2, average = 'binary')
})

In [ ]:
GradBoost_scores= pd.DataFrame(GB)
GradBoost_scores

### 7(e) K-Nearest Neighbours with Hyper Parameter tuning

In [ ]:
#Standardize the data

scaler = StandardScaler()
scaler.fit(X_train)
X_train_KNN = scaler.transform(X_train)
X_test_KNN = scaler.transform(X_test)
y_train_KNN = y_train
y_test_KNN = y_test



In [ ]:
#knn
k_fold = 10
param_grid = {
    'n_neighbors': list(range(1,round(len(y_train)**0.5),2)),
    'metric': ['euclidean', 'cosine']    
}


gridSearch = GridSearchCV(KNeighborsClassifier(), param_grid, cv=k_fold, scoring='recall',
                              n_jobs=-1)  # n_jobs=-1 will utilize all available CPUs 
gridSearch.fit(X_train_KNN, y_train)
print('score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)
best_KNN = gridSearch.best_estimator_
y_pred_KNN = best_KNN.predict(X_test_KNN)


In [ ]:
KNN = []
for k in range(1,int(len(y_test)**0.5)*2,2):
    knn = KNeighborsClassifier(n_neighbors=k,  metric='cosine')
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    
    KNN.append({ 
                'k':k,
                'Accuracy': accuracy_score(y_test, y_pred_knn),
                'Precision': precision_score(y_test, y_pred_knn),
                'Recall':recall_score(y_test, y_pred_knn),
                'F1': f1_score(y_test,y_pred_GB),
                'F2': fbeta_score(y_test,y_pred_GB, beta = 2, average = 'binary')
        })


In [ ]:
KNN_scores_df = pd.DataFrame(KNN)
KNN_scores_df

In [ ]:
KNN_scores_df.loc[KNN_scores_df['Recall'].idxmax()]

In [ ]:
KNN = []

KNN.append({ 
                'k':k,
                'Accuracy': accuracy_score(y_test, y_pred_knn),
                'Precision': precision_score(y_test, y_pred_knn),
                'Recall':recall_score(y_test, y_pred_knn),
                'F1': f1_score(y_test,y_pred_GB),
                'F2': fbeta_score(y_test,y_pred_GB, beta = 2, average = 'binary')
        })

In [ ]:
KNN = []

KNN.append({ 
                'Model':'KNN',
                'Accuracy': 0.816265,
                'Precision': 0.691610,
                'Recall':0.738499,
                'F1': 0.785714,
                'F2': 0.777669
        })

In [ ]:
KNN_scores_df_1 = pd.DataFrame(KNN)
KNN_scores_df_1

### 7(f) XG Boost

In [ ]:

xgboost = XGBClassifier(random_state=99)
_ = xgboost.fit(X_train, y_train)


In [ ]:
y_pred_XGB = xgboost.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred_XGB)

In [ ]:
XGB = []
XGB.append({    'Model' : 'XGBoost',
                'Accuracy': accuracy_score(y_test, y_pred_XGB),
                'Precision': precision_score(y_test, y_pred_XGB),
                'Recall':recall_score(y_test, y_pred_XGB),
                'F1': f1_score(y_test,y_pred_XGB),
                'F2': fbeta_score(y_test,y_pred_XGB, beta = 2, average = 'binary')
})

In [ ]:
XGBoost_scores= pd.DataFrame(XGB)
XGBoost_scores

### 7(g) Logistic Regression with Hyper parameter tuning

In [ ]:
param_grid_LR = { 'solver': ['newton-cg', 'lbfgs', 'liblinear','sag', 'saga'],
                      'penalty': ['l1', 'l2', 'none'],
                      'C': [100, 10, 1.0, 0.1, 0.01],
                      'max_iter': [1000] 
                     }

In [ ]:
best_logClassifier = GridSearchCV(estimator=LogisticRegression(random_state=99),
                                    scoring= 'recall', param_grid=param_grid_LR, 
                                    cv=2, verbose=0,  n_jobs = -1)
best_logClassifier = best_logClassifier.fit(X_train, y_train)

In [ ]:
y_pred_LR = best_logClassifier.predict(X_test)

In [ ]:
LogReg = []
LogReg.append({ 'Model' : 'Logistic Regression', 
                'Accuracy': accuracy_score(y_test, y_pred_LR),
                'Precision': precision_score(y_test, y_pred_LR),
                'Recall':recall_score(y_test, y_pred_LR),
                'F1': f1_score(y_test,y_pred_LR),
                'F2': fbeta_score(y_test,y_pred_LR, beta = 2, average = 'binary')
})

In [ ]:
LogReg_scores= pd.DataFrame(LogReg)
LogReg_scores

### 7(h) Artificial Neural Networks(ANN)

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(200), (200,100), (200,100,50), (200,100,50,40)], 
    'solver': ['lbfgs','adam', 'sdg'], 
    'activation': ['tanh','relu'],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

In [ ]:
best_grid_search_model = GridSearchCV(estimator=MLPClassifier(max_iter=100), 
                                    scoring='recall', param_grid=param_grid)
_ = best_grid_search_model.fit(X_train, y_train)

In [ ]:
y_pred_ANN = best_grid_search_model.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred_ANN)

In [ ]:
ANN = []
ANN.append({    'Model' : 'Neural Network', 
                'Accuracy': accuracy_score(y_test, y_pred_ANN),
                'Precision': precision_score(y_test, y_pred_ANN),
                'Recall':recall_score(y_test, y_pred_ANN),
                'F1': f1_score(y_test,y_pred_ANN),
                'F2': fbeta_score(y_test,y_pred_ANN, beta = 2, average = 'binary')
})

In [ ]:
ANN_scores= pd.DataFrame(ANN)
ANN_scores

In [ ]:
best_grid_search_model.get_params()

In [ ]:
ann = MLPClassifier(hidden_layer_sizes=(250,150,50), solver='lbfgs', max_iter=150)
_ = ann.fit(X_train, y_train)
y_pred_ANN_1 = ann.predict(X_test)

In [ ]:
ANN_1 = []
ANN_1.append({  'Model' : 'Neural Network-Final', 
                'Accuracy': accuracy_score(y_test, y_pred_ANN_1),
                'Precision': precision_score(y_test, y_pred_ANN_1),
                'Recall':recall_score(y_test, y_pred_ANN_1),
                'F1': f1_score(y_test,y_pred_ANN_1),
                'F2': fbeta_score(y_test,y_pred_ANN_1, beta = 2, average = 'binary')
})

In [ ]:
ANN_1_scores= pd.DataFrame(ANN_1)
ANN_1_scores

# Step-8: Performance comparision

In [ ]:
final = ANN_1_scores.append(ANN_scores).append(XGBoost_scores).append(Decision_Tree_scores).append(Random_Forest_scores).append(AdaBoost_scores).append(GradBoost_scores).append(LogReg_scores).append(KNN_scores_df_1)
#final = pd.concat(XGBoost_scores,ANN_1_scores,ANN_scores)
       


In [ ]:
final

# Step-9: Conclusion

Below is the summary of the analysis:
    
Broadly, the attributes can be classified into three categories:
    1) Student attributes collected at the time of student enrolment into the college
    2) Student academic performance for the first year(2 semesters)
    3) Macro-economic data like Inflation, GDP, Unemployment.
    
Regarding the macro-economic data ("Employment rate", "GDP" and "Inflation), there is ambiguity on the year considered (whether it was at the time of enrolment or at the time of student drop-out).
Hence, the same are being dropped for the analysis.

We considered two cases for our analysis:
    
In Case-1, we considered only the student attributes collected at the time of enrolment. 
In Case-2, we considered all the attributes excluding the macro-economic data.

The primary objective is to identify the drop-outs early in their academic path so that necessary inteventions by the University can be put into place. So, we aim to minimize the False Negatives through our models. Accordingly, the score we have to maximize is 'Recall'.

For the first case, we applied two models to estimate whether we get good performance or not with just the student attributes collected at the time of enrolment so that the University can focus on the potential drop-outs  right from the beginning of the semester. Since, we did not get satisfactory results, we concluded that first year academic performance attributes are crucial to have better results. Accordingly, we considered Case-2 wherein we included all the attributes(excluding the macro-economic data).

Below is the summary of the performance of all models:


| Model      | Accuracy | Precision | Recall | F1 | F2 |
| :----:     |    :----:   |   :----:  |  :----:  |  :----:  |  :----:  |
| Neural Network | 0.865964 | 0.779097 | 0.794189  | 0.786571 | 0.791124 |
| XGBoost | 0.866717 | 0.796482 | 0.767554 | 0.781751 | 0.773171 |
| Decision Tree |0.850904 | 0.745995 | 0.789346 | 0.767059 | 0.780278 |
| Random Forest | 0.871235 | 0.837989 | 0.726392 | 0.778210 | 0.746269 |
| AdaBoost | 0.852410 | 0.750577 | 0.786925 | 0.768322 | 0.779376 |
| Gradient Boost | 0.866717 | 0.795000 | 0.769976 | 0.782288 | 0.774854 |
| Logistic Regression | 0.857681 | 0.764151 | 0.784504 | 0.774194 | 0.780347 |
| KNN | 0.816265 | 0.691610 | 0.738499 | 0.785714 | 0.77769 |


Considering F2 score and recall, Neural Network model performs better than all the models. Also, the accuracy of ANN is comparable to the best accuracy acheived. So, ANN is the best model in this case. A recall of 79.4% in ANN means that the model is able to identify close to 80 out of 100 drop-out students correctly. 

The novelty and the simplicity of this model is that with almost no cost(i.e., only with the basic, available information of the students) the University can identify correctly close to 80 out of 100 potential drop-out students. The university can leverage this information to conduct counselling/mentoring sessions with students, offer alternate class scheduling, recommend courses, tailor the curriculum etc., to retain these students. This way, it is win-win for both the student and the University. Further, it will benefit society at large owing to the postive correlation of tertiary education with the countries'economy.



In [ ]:
pip install nbconvert



In [ ]:
conda install nbconvert


In [ ]:
!jupyter nbconvert --to html Student Dropout Prediction.ipynb
